In [1]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict
import ts_code.nsfg as nsfg
import ts_code.thinkstats2 as thinkstats2
import ts_code.thinkplot as thinkplot
import ts_code.first as first
import matplotlib.pyplot as plt
import ts_code.brfss as brfss
%matplotlib inline

## Chapter 11 - Regression

**Regression** is the problem of fitting any kind of model to any kind of data. The goal is to describe the relationship between one set of variables, called **dependent variables** and another set of variables, called **independent variables**.

When there is only one dependent and one explanatory variable, that's **simple regression**.

Regression with more than one explanatory variable is **multiple regression**, if there is more than one dependent variable, that is **multivariate regression**.

If the relationship between the dependent and explanatory variable is linear, that's **linear regression**.

If $y$ is the dependent, and $x_{1}$, $x_{2}$ are explanatory variables, the following is a linear regression model;

$$y = \beta_{0} + \beta_{1}x_{1} + \beta_{2}x_{2} + \epsilon$$

where $\beta_{0}$ is the intercept, $\beta_{1}$ and $\beta_{2}$ are parameters and $\epsilon$ is the residual due to random variation or other unknown factors.

Given a sequence of $y$, $x_{1}$, and $x_{2}$ values we can find parameters $\beta_{0}$, $\beta_{1}$, and $\beta_{2}$ that minimize the sum of $\epsilon^{2}$. This called **ordinary least squares**.

**StatsModels** is  a Python package that provides several forms of regression and other analyses.

In [2]:
import statsmodels.formula.api as smf

In [3]:
#model from least squares
live, firsts, others = first.MakeFrames()
formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()

`statsmodels` provides two interfaces (APIs); the "formula" API uses strings to identify the dependent and explanatory variables. It uses a syntax called *patsy*; in this example, the **~** operator separates the dependent variable on the left from the explanatory variables on the right.

`smf.ols` takes the formula string and the DataFrame, live, and returns an OLS object that represents the model. The name ols stands for **ordinary least squares**.

The `fit` method fits the model to the data and returns a *RegressionResults* object that contains the results.

The results are available as attributes. `params` is a Series that maps from variable names to their parameters;

In [4]:
inter = results.params['Intercept']
slope = results.params['agepreg']

In [5]:
for key, val in results.params.items():
    print(key, val)

Intercept 6.83039697331
agepreg 0.0174538514718


In [6]:
slope_pvalues = results.pvalues['agepreg']
slope_pvalues

5.7229471073134105e-11

In [7]:
for key, val in results.pvalues.items():
    print(key, val)

Intercept 0.0
agepreg 5.72294710731e-11


In [8]:
r2 = results.rsquared
r2

0.0047381154747102583

`results` also provides *f_pvalue*, which is the p-value associated with the model as a whole, similar to testing whether R2 is statistically significant.

In [9]:
f_p = results.f_pvalue
f_p

5.7229471072556979e-11

`Results` also provides *resid*, a sequence of residuals, and *fittedvalues*, a
sequence of fitted values corresponding to agepreg.

In [10]:
res = results.resid
y_hat = results.fittedvalues
res

0        1.403333
1        0.359539
2        2.044489
3       -0.141599
4       -0.962826
5        1.260849
6        2.228908
7        1.018195
8        0.241999
9       -0.769680
10       0.532666
11      -0.231836
12      -3.259413
15       0.362635
16       0.140228
17      -0.847949
19       1.432466
20       0.823364
21      -1.597949
23      -0.468745
24       0.095166
25      -0.531215
26       0.724560
27      -0.034053
28      -1.131461
29       0.229053
31       0.592230
32       0.211439
33      -0.067534
34      -0.355553
           ...   
13546    1.555910
13547   -3.028293
13548    2.814781
13551   -1.896107
13553    0.193986
13555   -1.004905
13556   -0.074720
13557   -0.789957
13559    0.633041
13560    0.574920
13561   -0.446897
13562    0.976070
13563    0.489176
13564    0.387405
13565    0.941163
13566    0.099560
13569   -1.332066
13570   -0.568945
13571   -1.316138
13572   -1.596667
13573   -0.656245
13574   -1.131445
13576   -0.945486
13578   -1.249289
13579   -0

In [11]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Thu, 22 Feb 2018   Prob (F-statistic):           5.72e-11
Time:                        19:44:42   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.0

We saw that first babies tend to be lighter, we also saw that age has affect on birth weight. These could be related, but let's take a quick look before moving on to a **multiple regression** for the two variables.

In [12]:
diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_weight

-0.12476118453549034

In [13]:
diff_age = firsts.agepreg.mean() - others.agepreg.mean()
diff_age

-3.586434766150152

In [14]:
results = smf.ols('totalwgt_lb ~ agepreg', data=live).fit()
slope = results.params['agepreg']
slope

0.017453851471802843

So, the difference in age is about 3.5 years, the difference in weight is .124 pounds and we know each year adds 0.0175 pounds to the birth weight.

In [15]:
slope * diff_age

-0.062597099721694707

So, this looks like about half of the difference being accounted for by the mother's age.

In [16]:
live['isfirst'] = live.birthord == 1 #add boolean indicator column
formula = 'totalwgt_lb ~ isfirst'
results = smf.ols(formula, data=live).fit()

In [17]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            totalwgt_lb   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     17.74
Date:                Thu, 22 Feb 2018   Prob (F-statistic):           2.55e-05
Time:                        19:44:42   Log-Likelihood:                -15909.
No. Observations:                9038   AIC:                         3.182e+04
Df Residuals:                    9036   BIC:                         3.184e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.3259      0.021    3

The **ols** treats the isfirst column as a **categorical variable**. The estimated parameter on weight when isfirst is true is -.0125 pounds, which is the difference in birth weight between first babies and others.

The slope and the intercept are statistically significant, which means that they were unlikely to occur by chance, but the the R2 value for this model is small, which means that isfirst doesn't account for a substantial part of the variation in birth weight

In [18]:
formula = 'totalwgt_lb ~ agepreg'
results = smf.ols(formula, data=live).fit()

In [19]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Thu, 22 Feb 2018   Prob (F-statistic):           5.72e-11
Time:                        19:44:43   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.0

Agepreg results in a similary small $R^{2}$ with statistically significant parameters.

In [20]:
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()

In [21]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     24.02
Date:                Thu, 22 Feb 2018   Prob (F-statistic):           3.95e-11
Time:                        19:44:43   Log-Likelihood:                -15894.
No. Observations:                9038   AIC:                         3.179e+04
Df Residuals:                    9035   BIC:                         3.182e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.9142      0.078     

Previously we have noted the agepreg is nonlinear, so we might want to add a variable to caputre more of this relationship.

A common choice is to add a variable that represents the squares of the ages.

In [22]:
live['agepreg2'] = live.agepreg**2
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results = smf.ols(formula, data=live).fit()

In [23]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            totalwgt_lb   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     22.64
Date:                Thu, 22 Feb 2018   Prob (F-statistic):           1.35e-14
Time:                        19:44:43   Log-Likelihood:                -15884.
No. Observations:                9038   AIC:                         3.178e+04
Df Residuals:                    9034   BIC:                         3.181e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.6923      0.286     

We can see the coefficient of agepreg2 is negative, so this parabola curve downward. This model account for more variability in birth weight and results in a smaller parameter for isfirst, making it no longer statistically significant.

We conclude that the apparent difference in birth weight is explained, at least in part, by the difference in mother's age. When we include mother's age in the model, the effect of isfirst gets smaller, and the remaining effect might be due to chance.

This process is still considered linear regression, because the dependent variable is a linear function of the explanatory variables, regardless of whether some variables are nonlinear functions of others.

In this example, mother's age acts as a **control variable**.

When we include *agepreg* in the model it "controls for" the difference in age between first-time mothers and others, making it possible to isolate the effect (if any) of *isfirst*.

In [62]:
#using pandas to perform a SQL join operation

live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid #set index of respondents to the caseid
join = live.join(resp, on='caseid', rsuffix='_r') #add suffix for duplicate cols

In [68]:
t = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7: #check that data has some variability
            continue
        #include agepreg in formula, we know it has predictive power
        formula = 'totalwgt_lb ~ agepreg + ' + name
        model = smf.ols(formula, data=join)
        if model.nobs < len(join)/2: #check that number of observations 
            #is greater than half the length of a column.
            continue
        results = model.fit() #fit model
        
    except (ValueError, TypeError):
        continue
    t.append((results.rsquared, name)) #appended the r2 for each variable

In [69]:
t[:5]

[(0.0053576473236406352, 'caseid'),
 (0.0057500139850770182, 'pregordr'),
 (0.0063309802373902047, 'pregend1'),
 (0.016017752709788113, 'nbrnaliv'),
 (0.005543156193094867, 'cmprgend')]

In [70]:
#sort and find highest R2 values
t.sort(reverse=True)
for mse, name in t[:30]: #print 30 highest R2 values
    print(name, mse)

totalwgt_lb 1.0
birthwgt_lb 0.949812730598
lbw1 0.300824078447
prglngth 0.130125194886
wksgest 0.123400413634
agecon 0.102031499282
mosgest 0.0271442746396
babysex 0.0185509252939
race_r 0.0161995035863
race 0.0161995035863
nbrnaliv 0.0160177527098
paydu 0.0140037955781
rmarout03 0.0134300664657
birthwgt_oz 0.0131024576157
anynurse 0.0125290225418
bfeedwks 0.0121936884045
totincr 0.0118700690312
marout03 0.0118078019944
marcon03 0.0117525993544
cebow 0.0114377709196
rmarout01 0.0114077371386
rmarout6 0.0113541384728
marout01 0.0112693572468
hisprace_r 0.011238349302
hisprace 0.011238349302
mar1diss 0.0109615635908
fmarcon5 0.0106049646843
rmarout02 0.0105469132066
marcon02 0.0104814017955
fmarout5 0.0104616913674


Sometimes you start with a theory and use data to test it. Other times you start with data and go looking for possible theories. The second approach, which this section demonstrates, is called **data mining**. 

An advantage of data mining is that it can discover unexpected patterns. A hazard is that many of the patterns it discovers are either random or spurious.

In [71]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 +' 
           'nbrnaliv>1 + paydu==1 + totincr')
#C(race) says to treat the numerical data in the race column as categorical

#encoding for babysex is 1 for male, 2 for female; writing babysex==1
#converts it to boolean, True for male and false for female.

#nbrnaliv>1 is True for multiple births
#paydu==1 is True for respondents who own their houses.

results = smf.ols(formula, data=join).fit()

In [72]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            totalwgt_lb   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     79.98
Date:                Thu, 22 Feb 2018   Prob (F-statistic):          4.86e-113
Time:                        20:12:59   Log-Likelihood:                -14295.
No. Observations:                8781   AIC:                         2.861e+04
Df Residuals:                    8773   BIC:                         2.866e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                6.6303 

Linear regression can be generalized to handle other kinds of dependent variables. If the dependent variable is boolean, the generalized model is called **logistic regression**. If the dependent variable is an integer count, it's called **Poisson regression**.

If we wanted to use linear regression to predict a boolean, we might encode True as 1, False as 0. This produces predictions that are hard to predict, like $y=0.5$. It's tempting to read this as a probability, but it isn't, this model could also output $y=1.1$ or $y=-0.1$.

Logistic regression avoids this problem by expressing predictions in terms of odds rather than probabilities. Odds of an event is the ratio of the probability it will occur to the probability
that it will not.

So if I think my team has a **75% chance** of winning, I would say that the odds in their favor are **three to one**, because the chance of winning is three times the chance of losing.

Easily convert between odds and probabilities;
$$\text{odds} = \frac{p}{(1-p)}$$

or;

$$p = \frac{\text{odds}}{(\text{odds}+1)}$$


Logistic regression is based on the following model;

$$log(o) = \beta_{0} + \beta_{1}x_{1} + \beta_{2}x_{2} + \epsilon$$

where $o$ is the odds in favor of a particular outcome.

Unlike linear regression, logistic regression does not have a closed form solution, so it is solved by guessing an initial solution and improving it  iteratively.

The usual goal is to find the maximum-likelihood estimate (MLE), which is the set of parameters that maximizes the likelihood of the data.

In [79]:
#intial data
y = np.array([0, 1, 0, 1])
x1 = np.array([0, 0, 0, 1])
x2 = np.array([0, 1, 1, 1])

#starting guesses
beta = [-1.5, 2.8, 1.1]

#compute each row
log_o = beta[0] + beta[1] * x1 + beta[2] * x2

In [83]:
#convert log odds to odds
o = np.exp(log_o)
#convert odds to probabilities
p = o / (o+1)

The likelihood of an outcome is $p$ when $y=1$ and $1-p$ when $y=0$. For example, if we think the probability of a boy is 0.8 and the outcome is a boy, the likelihood is 0.8; if the outcome is a girl, the likelihood is 0.2. 

We can compute the likelihoods and then the over **likelihood** of the data is the product of the likelihoods.

In [84]:
likes = y * p + (1-y) * (1-p)

In [85]:
like = np.prod(likes)
like

0.18009335296730339

For these values of *beta*, the likelihood of the data is 0.18. 

The goal of logistic regression is to find parameters that maximize this likelihood. To do that, most statistics packages use an iterative solver like Newton's method.

`StatsModels` provides an implementation of logistic regression called *logit*,
named for the function that converts from probability to log odds.

*logit* requires the dependent variable to be binary (rather than boolean), so we can use *astype(int)* to convert to binary integers:

In [87]:
live['boy'] = (live.babysex==1).astype(int)

In [89]:
model = smf.logit('boy ~ agepreg', data=live)
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.693015
         Iterations 3
                           Logit Regression Results                           
Dep. Variable:                    boy   No. Observations:                 8884
Model:                          Logit   Df Residuals:                     8882
Method:                           MLE   Df Model:                            1
Date:                Thu, 22 Feb 2018   Pseudo R-squ.:               6.144e-06
Time:                        20:30:30   Log-Likelihood:                -6156.7
converged:                       True   LL-Null:                       -6156.8
                                        LLR p-value:                    0.7833
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0058      0.098      0.059      0.953      -0.185       0.197
agepreg        0.0010      0.

The result is a Logit object that represents the model. It contains attributes called `endog` and `exog` that contain the **endogenous variable**, another name for the dependent variable, and the **exogenous variables**, another name for the explanatory variables.

In [95]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

The parameter of *agepreg* is positive, which suggests that older mothers are more likely to have boys, but the p-value is 0.783, which means that the apparent effect could easily be due to chance.

The coefficient of determination, $R^{2}$, does not apply to logistic regression, but there are several alternatives that are used as "pseudo $R^{2}$ values." These values can be useful for comparing models. 

For example, here's a model that includes several factors believed to be associated with sex ratio:

In [91]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.692944
         Iterations 3


In [92]:
print(results.summary())

                           Logit Regression Results                           
Dep. Variable:                    boy   No. Observations:                 8782
Model:                          Logit   Df Residuals:                     8776
Method:                           MLE   Df Model:                            5
Date:                Thu, 22 Feb 2018   Pseudo R-squ.:               0.0001440
Time:                        20:33:52   Log-Likelihood:                -6085.4
converged:                       True   LL-Null:                       -6086.3
                                        LLR p-value:                    0.8822
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0301      0.104     -0.290      0.772      -0.234       0.173
C(race)[T.2]    -0.0224      0.051     -0.439      0.660      -0.122       0.077
C(race)[T.3]    -0.0005      0.083     -0.00

To check our work, we compute the accuracy of our model.

In the NSFG data, there are more boys than girls, so the baseline statregy is to guess boy each time.

In [96]:
actual = endog['boy']
baseline = actual.mean()

In [97]:
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)

*results.predict* returns a NumPy array of probabilities, which we round off to 0 or 1. 

Multiplying by actual yields 1 if we predict a boy and get it right, 0 otherwise. 

So, true_pos indicates "true positives".

Similarly, true_neg indicates the cases where we guess "girl" and get it right.

In [98]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
acc

0.51150079708494645

The result is 0.512, slightly better than the baseline, 0.507. 

But, you should not take this result too seriously. We used the same data to build and test the model, so the model may not have predictive power on new data.

In [106]:
#predict sex of a friend who is 35, with a 39 year old husband
#and its their third kid
columns = ['agepreg', 'hpagelb', 'birthord', 'race']
new = pd.DataFrame([[35, 39, 3, 2]], columns=columns)
y = results.predict(new)
print('boy') if y[0]>0.5 else print('girl')

boy


### Exercises

**Exercise 11.1** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [129]:
t = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7: 
            continue
            
        formula = 'prglngth ~ ' + name
        model = smf.ols(formula, data=join)
        if model.nobs < len(join)/2: 
            continue
        results = model.fit() #fit model
        
    except (ValueError, TypeError):
        continue
    t.append((results.rsquared, name)) 

In [130]:
t.sort(reverse=True)
t = [col for col in t if col[1][-1] != 'i'] #remove indicator cols
for mse, name in t[:30]: #print 30 highest R2 values
    print(name, mse)

prglngth 1.0
wksgest 0.806243411614
totalwgt_lb 0.124457431481
birthwgt_lb 0.119773078049
lbw1 0.103725422046
mosgest 0.0956243198959
canhaver 0.0060504952682
nbrnaliv 0.00457756578553
anynurse 0.00245202488371
bfeedwks 0.00236918394467
pregend1 0.0022493894338
cmlastlb_r 0.0020431424422
cmlastlb 0.0020431424422
evuseint 0.00189175277586
paydu 0.00187682190302
anymschp 0.00177999843325
gestasun_m 0.00165713195501
hlpmc 0.00161252106165
diabetes 0.001600707227
ptsb4mar 0.00158691214383
cmhivtst 0.00148078379171
todayspg 0.00147583725852
marbefhx 0.00144127783259
widrawal 0.00141412298374
liv1chld 0.00141084482997
methdiss 0.00140159126169
sest_r 0.00132236819817
sest 0.00132236819817
manrel 0.00131095845942
matchfound 0.00130910737713


In [122]:
#nbrnaliv number of live births
#paydu rent or own

In [123]:
formula = ('prglngth ~ nbrnaliv>1 + paydu==1')

#nbrnaliv>1 is True for multiple births
#paydu==1 is True for respondents who own their houses.

results = smf.ols(formula, data=join).fit()

In [125]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               prglngth   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     46.65
Date:                Thu, 22 Feb 2018   Prob (F-statistic):           6.98e-21
Time:                        21:07:10   Log-Likelihood:                -18252.
No. Observations:                8884   AIC:                         3.651e+04
Df Residuals:                    8881   BIC:                         3.653e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               38.8430 

**Exercise 11.2** The Trivers-Willard hypothesis suggests that for many mammals the sex ratio depends on "maternal condition"; that is, factors like the
mother's age, size, health, and social status. 

Some studies have shown this effect among humans, but results are mixed.

In this chapter we tested some variables related to these factors, but didn't find any with a statistically significant effect on sex ratio.

As an exercise, use a data mining approach to test the other variables in the pregnancy and respondent files. Can you find any factors with a substantial
effect?

In [167]:
t = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7: 
            continue
            
        formula = 'boy ~ ' + name
        try:
            model = smf.logit(formula, data=live)
            nobs = len(model.endog)
            if nobs < len(live)/2:
                continue
            results = model.fit() #fit model

        except:
            continue

    except (ValueError, TypeError):
        continue
    t.append((results.prsquared, name))

Optimization terminated successfully.
         Current function value: 0.692996
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692962
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692850
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693001
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692903
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692766
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.693019
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692995
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693017
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692989
  

Optimization terminated successfully.
         Current function value: 0.692895
         Iterations 5
         Current function value: 0.692784
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.692643
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.693019
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692840
         Iterations 5


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Optimization terminated successfully.
         Current function value: 0.692975
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692989
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692975
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693002
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693019
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692978
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692977
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692815
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693018
  

In [168]:
t.sort(reverse=True)
t = [col for col in t if col[1][-1] != 'i'] #remove indicator cols
for mse, name in t[:30]: #print 30 highest R2 values
    print(name, mse)

totalwgt_lb 0.00967200535184
birthwgt_lb 0.0092489816712
lbw1 0.00104859533196
frsteatd 0.0007600842059
fmarout5 0.000681996122597
brnout 0.000641068785277
bpa_bdscheck1 0.000635132392883
rmarout6 0.000620359718917
pmarpreg 0.000525037533603
cmprgbeg 0.000385150531009
cmintstr 0.000347849734469
fmarcon5 0.000305032394269
cmintfin 0.000254556542944
fmarital 0.000250894216646
pregend1 0.000244550097046
evuseint 0.000226956417488
cmbirth 0.000208672062879
pregnum 0.000207028475367
agescrn 0.000202205053819
ager 0.000202205053819
wantbold 0.000199452287779
cmprgend 0.000167794737359
cmbabdob 0.000167794737359
datecon 0.000160855399429
datend 0.000158255290499
prglngth 0.000141756277198
anyusint 0.000139420326087
religion 0.000130989904309
oldwantp 0.000112175099662
wantpart 0.000107334273798


In [170]:
formula='boy ~ agepreg + fmarout5==5 + infever==1'
model = smf.logit(formula, data=live)
results = model.fit()
results.summary() 

PatsyError: Error evaluating factor: NameError: name 'infever' is not defined
    boy ~ agepreg + fmarout5==5 + infever==1
                                  ^^^^^^^^^^